In [187]:
import pymongo
from collections import Counter

In [188]:
# ssh -N -oHostKeyAlgorithms=+ssh-dss -f -L 8848:222.20.95.102:8848 -p 3000 chensy@222.20.95.101

In [189]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:8848",
                          username="admin_user",
                          password='admin_password')
mydb = myclient['MSC']

In [190]:
mydb.list_collection_names()

['samples', 'projects']

In [153]:
list( mydb['samples'].find({"ProjectId": "PRJNA613685"}) )

[{'_id': ObjectId('64c46ff84a64121ab73aa42c'),
  'Source': ['Bone marrow'],
  'Marker': ['LEPRhiCD45low'],
  'Treatment': ['identified LEPRhiCD45low BM-MSCs within the CD271+ BM-MNC population'],
  'Condition': 'osteoporosis',
  'LibraryStrategy': 'scRNA-Seq',
  'Species': 'Homo sapiens',
  'SampleId': 'SRS6346968',
  'ExperimentId': 'SRX7960930',
  'ProjectId': 'PRJNA613685',
  'RunId': 'SRR11359429',
  'Comment': 'GSM4423510: MNC_osteoporosis; Homo sapiens; RNA-SeqGSM4423510 None',
  'Gender': 'Unknown',
  'Instrument': 'Illumina NovaSeq 6000',
  'LibraryLayout': 'PAIRED',
  'LibrarySelection': 'cDNA',
  'Length': 300,
  'Spots': 140.55,
  'Url': ['era-fasp@fasp.sra.ebi.ac.uk:/vol1/fastq/SRR113/029/SRR11359429/SRR11359429_1.fastq.gz',
   'era-fasp@fasp.sra.ebi.ac.uk:/vol1/fastq/SRR113/029/SRR11359429/SRR11359429_2.fastq.gz']},
 {'_id': ObjectId('64c46ff84a64121ab73aa42e'),
  'Source': ['Bone marrow'],
  'Marker': ['LEPRhiCD45low'],
  'Treatment': ['identified LEPRhiCD45low BM-MSCs wi

In [158]:
mydb['samples'].delete_many({"ProjectId": "PRJNA928451", "LibraryStrategy": 'MNase-seq'}) 

In [115]:
mydb['samples'].update_many({"ProjectId": "PRJNA488091"}, { '$set': { 'Treatment': ['chondrogenically pre-cultured'], 'Source': ['Bone marrow'] } }) 

In [114]:
mydb['Projects'].update_many({"ProjectId": "PRJNA488091"}, { '$set': { 'Treatment': ['chondrogenically pre-cultured'], 'Source': ['Bone marrow'] } }) 

## Change to single cell

# Change sequencing type

In [99]:
def change(pid, tp):
    mydb['projects'].update_one({"ProjectId": pid}, { "$set": { 'SequencingType': [tp] } })
    mydb['samples'].update_many({"ProjectId": pid}, { '$set':{ 'LibraryStrategy': tp } })

In [152]:
change('PRJNA823365', 'scRNA-Seq')

# Distingush EV and MSC

In [94]:
mydb['projects'].update_many({}, { '$set': { 'Target': 'MSC' } })

In [107]:
pids = """PRJNA956270
PRJNA929917
PRJNA863244
PRJNA812533
PRJNA801605
PRJNA745950
PRJNA326339
PRJNA888211
PRJNA705354
PRJNA1043587
PRJDB9427
PRJNA639151
PRJNA287115""".split('\n')

In [108]:
mydb['projects'].update_many( { 'ProjectId': { '$in': pids } }, { '$set': { 'Target': 'EV/Exosome' } } )

# TODO 

好像有些没有Sequencing type

In [197]:
todo = list( mydb['projects'].find({
    'ExpressionDown': { "$ne": True },
    'Target': 'MSC',
    'SequencingType': { '$exists': False }
},
   {'ProjectId': 1, 'Summary': 1, 'SampleCount':1, '_id': 0}
    
))

In [198]:
len(todo)

47

In [199]:
todo

[{'ProjectId': 'PRJNA860821',
  'Summary': 'To elucidate how the core circadian component BMAL1 and clock regulate the osteogenic capacity of human bone marrow MSCs, RNA-seq was performed to study the impact of diminishing core component BMAL1 or CLOCK in MSCs by lentiviruses.\nOverall design: Examination of the mRNA profiles in human bone marrow MSCs with diminishing  BMAL1 or CLOCK during osteogenic differentiation Less...',
  'SampleCount': 9},
 {'ProjectId': 'PRJEB42334',
  'Summary': 'Human adipose-derived mesenchymal stem cells were cultured either in hypoxia (Hx; <0.1% oxygen) or standard culture conditions (normoxia, Nx) over 48 hours in serum-free medium. Human tympanic membrane keratinocytes were cultured in standard culture conditions over 48 hours in serum-free medium.',
  'SampleCount': 55},
 {'ProjectId': 'PRJNA483407',
  'Summary': 'Human bone marrow-derived mesenchymal stem/stromal cells (hBM MSCs) have multiple functions, critical for skeletal formation and function. T

In [193]:
for item in todo:
    print(f"snakemake -j 24 -p -s EXPRESSION.smk --config pid={item['ProjectId']}")